In [2]:
import logging
from simtk.openmm import app, Platform, unit
from simtk.openmm import VerletIntegrator, AndersenThermostat, MonteCarloBarostat
from simtk.openmm.app import Simulation, StateDataReporter
from mdtraj.reporters import HDF5Reporter

import openmmtools.testsystems as testsystems
import os

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

class SimulationSetup:
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    time_step = 2 * unit.femtoseconds
    equil_steps = 50000

def run_simulation(platform_name, file_path, file_name, constrain_all_bonds=True, report_production=True, num_steps=100000, write_out_freq=1000, **kwargs):
    logger.info("Starting simulation via OpenMM")

    # Load WaterBox test system from openmmtools
    water_box = testsystems.WaterBox(box_edge=3.0 * unit.nanometer)

    platform = Platform.getPlatformByName(platform_name)
    system = water_box.system

    path = '{}/{}.h5'.format(file_path, file_name)

    constrain_what_bond = app.AllBonds if constrain_all_bonds else app.HBonds
    system = water_box.system
    system = water_box.topology.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer, constraints=constrain_what_bond, rigidWater=False)

    logger.info("Adding thermostat and barostat")
    thermostat = AndersenThermostat(SimulationSetup.temperature, 1/unit.picosecond)
    system.addForce(thermostat)
    barostat = MonteCarloBarostat(SimulationSetup.pressure, SimulationSetup.temperature)
    system.addForce(barostat)
    integrator = VerletIntegrator(SimulationSetup.time_step)

    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPeriodicBoxVectors(*water_box.positions)
    simulation.context.setPositions(water_box.positions)
    simulation.minimizeEnergy()
    logger.info("Energy minimized")

    # Equilibration
    if "equil_steps" in kwargs:
        SimulationSetup.equil_steps = kwargs["equil_steps"]
    logger.info(f"Equilibration steps: {SimulationSetup.equil_steps}")

    equil_reporter_path = "{}/equilibration_{}.dat".format(file_path, file_name)
    simulation.reporters.append(StateDataReporter(equil_reporter_path, SimulationSetup.equil_steps // 5000, step=True, volume=True, temperature=True, density=True))
    simulation.step(SimulationSetup.equil_steps)
    logger.info("Equilibration complete")

    state = simulation.context.getState(getPositions=True, getVelocities=True)
    water_box.positions = state.getPositions()
    water_box.velocities = state.getVelocities()
    water_box.box_vectors = state.getPeriodicBoxVectors()

    # Production
    del system
    del simulation

    system = water_box.topology.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer, constraints=constrain_what_bond)
    thermostat = AndersenThermostat(SimulationSetup.temperature, 1/unit.picosecond)
    system.addForce(thermostat)

    integrator = VerletIntegrator(SimulationSetup.time_step)
    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPeriodicBoxVectors(*water_box.box_vectors)
    simulation.context.setPositions(water_box.positions)

    if report_production:
        production_reporter_path = "{}/production_{}.dat".format(file_path, file_name)
        simulation.reporters.append(StateDataReporter(production_reporter_path, num_steps // 50000, step=True, potentialEnergy=True, temperature=True, density=True, kineticEnergy=True, totalEnergy=True, volume=True))

    simulation.reporters.append(HDF5Reporter(path, write_out_freq))
    simulation.step(num_steps)
    logger.info("Production complete")

    return os.path.abspath(path)

# Example usage
if __name__ == "__main__":
    platform_name = "CPU"  # or "CUDA", "OpenCL" depending on your hardware
    file_path = "./output"
    file_name = "waterbox_simulation"

    if not os.path.exists(file_path):
        os.makedirs(file_path)

    result_path = run_simulation(platform_name, file_path, file_name)
    print(f"Simulation results saved to: {result_path}")


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



AttributeError: 'Topology' object has no attribute 'createSystem'

In [9]:

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

class SimulationSetup:
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    time_step = 2 * unit.femtoseconds
    equil_steps = 50000

def run_simulation(platform_name, file_path, file_name, constrain_all_bonds=True, report_production=True, num_steps=100000, write_out_freq=1000, **kwargs):
    logger.info("Starting simulation via OpenMM")

    # Load WaterBox test system from openmmtools
    water_box = testsystems.WaterBox(box_edge=3.0 * unit.nanometer)

    platform = Platform.getPlatformByName(platform_name)
    system = water_box.system

    path = '{}/{}.h5'.format(file_path, file_name)

    constrain_what_bond = app.AllBonds if constrain_all_bonds else app.HBonds
    for force in system.getForces():
        if isinstance(force, NonbondedForce):
            force.setNonbondedMethod(NonbondedForce.PME)
            force.setCutoffDistance(1 * unit.nanometer)

    logger.info("Adding thermostat and barostat")
    thermostat = AndersenThermostat(SimulationSetup.temperature, 1 / unit.picosecond)
    system.addForce(thermostat)
    barostat = MonteCarloBarostat(SimulationSetup.pressure, SimulationSetup.temperature)
    system.addForce(barostat)
    integrator = VerletIntegrator(SimulationSetup.time_step)

    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPositions(water_box.positions)
    simulation.context.setVelocitiesToTemperature(SimulationSetup.temperature)
    simulation.minimizeEnergy()
    logger.info("Energy minimized")

    # Equilibration
    if "equil_steps" in kwargs:
        SimulationSetup.equil_steps = kwargs["equil_steps"]
    logger.info(f"Equilibration steps: {SimulationSetup.equil_steps}")

    equil_reporter_path = "{}/equilibration_{}.dat".format(file_path, file_name)
    simulation.reporters.append(StateDataReporter(equil_reporter_path, SimulationSetup.equil_steps // 5000, step=True, volume=True, temperature=True, density=True))
    simulation.step(SimulationSetup.equil_steps)
    logger.info("Equilibration complete")

    state = simulation.context.getState(getPositions=True, getVelocities=True, getEnergy=True, getForces=True, getPeriodicBoxVectors=True)
    water_box.positions = state.getPositions()
    water_box.velocities = state.getVelocities()
    box_vectors = state.getPeriodicBoxVectors()

    # Production
    del simulation

    system = water_box.system
    thermostat = AndersenThermostat(SimulationSetup.temperature, 1 / unit.picosecond)
    system.addForce(thermostat)

    integrator = VerletIntegrator(SimulationSetup.time_step)
    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPeriodicBoxVectors(*box_vectors)
    simulation.context.setPositions(water_box.positions)
    simulation.context.setVelocities(water_box.velocities)

    if report_production:
        production_reporter_path = "{}/production_{}.dat".format(file_path, file_name)
        simulation.reporters.append(StateDataReporter(production_reporter_path, num_steps // 50000, step=True, potentialEnergy=True, temperature=True, density=True, kineticEnergy=True, totalEnergy=True, volume=True))

    simulation.reporters.append(HDF5Reporter(path, write_out_freq))
    simulation.step(num_steps)
    logger.info("Production complete")

    return os.path.abspath(path)

# Example usage
if __name__ == "__main__":
    platform_name = "CPU"  # or "CUDA", "OpenCL" depending on your hardware
    file_path = "./output"
    file_name = "waterbox_simulation"

    if not os.path.exists(file_path):
        os.makedirs(file_path)

    result_path = run_simulation(platform_name, file_path, file_name)
    print(f"Simulation results saved to: {result_path}")


KeyboardInterrupt: 

In [24]:


# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

class SimulationSetup:
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    time_step = 2 * unit.femtoseconds
    equil_steps = 5000

def extend_box_z(positions, box_vectors, factor=5):
    """ Extend the box size in the z direction by the given factor """
    new_positions = []
    for i in range(factor):
        for pos in positions:
            new_pos = pos + unit.Quantity((0, 0, i * box_vectors[2][2].value_in_unit(unit.nanometer)), unit.nanometer)
            new_positions.append(new_pos)
    
    new_box_vectors = (
        box_vectors[0],
        box_vectors[1],
        box_vectors[2] * factor
    )
    return new_positions, new_box_vectors

def run_equilibration(platform_name, file_path, file_name, constrain_all_bonds=True, **kwargs):
    logger.info("Starting equilibration via OpenMM")

    # Load WaterBox test system from openmmtools
    water_box = testsystems.WaterBox(box_edge=3.0 * unit.nanometer)

    platform = Platform.getPlatformByName(platform_name)
    system = water_box.system

    path = '{}/{}.h5'.format(file_path, file_name)

    constrain_what_bond = app.AllBonds if constrain_all_bonds else app.HBonds
    for force in system.getForces():
        if isinstance(force, NonbondedForce):
            force.setNonbondedMethod(NonbondedForce.PME)
            force.setCutoffDistance(1 * unit.nanometer)

    logger.info("Adding thermostat and barostat")
    thermostat = AndersenThermostat(SimulationSetup.temperature, 1 / unit.picosecond)
    system.addForce(thermostat)
    barostat = MonteCarloBarostat(SimulationSetup.pressure, SimulationSetup.temperature)
    system.addForce(barostat)
    integrator = VerletIntegrator(SimulationSetup.time_step)

    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPositions(water_box.positions)
    simulation.context.setVelocitiesToTemperature(SimulationSetup.temperature)
    simulation.minimizeEnergy()
    logger.info("Energy minimized")

    # Equilibration
    if "equil_steps" in kwargs:
        SimulationSetup.equil_steps = kwargs["equil_steps"]
    logger.info(f"Equilibration steps: {SimulationSetup.equil_steps}")

    equil_reporter_path = "{}/equilibration_{}.dat".format(file_path, file_name)
    simulation.reporters.append(StateDataReporter(equil_reporter_path, SimulationSetup.equil_steps // 5000, step=True, volume=True, temperature=True, density=True))
    simulation.step(SimulationSetup.equil_steps)
    logger.info("Equilibration complete")

    return water_box,simulation

def run_production(platform_name, file_path, file_name, extended_positions, velocities, extended_box_vectors, water_box, report_production=True, num_steps=100000, write_out_freq=1000):
    logger.info("Starting production via OpenMM")

    path = '{}/{}.h5'.format(file_path, file_name)

    platform = Platform.getPlatformByName(platform_name)
    system = water_box.system

    thermostat = AndersenThermostat(SimulationSetup.temperature, 1 / unit.picosecond)
    system.addForce(thermostat)

    integrator = VerletIntegrator(SimulationSetup.time_step)
    simulation = Simulation(water_box.topology, system, integrator, platform)
    simulation.context.setPeriodicBoxVectors(*extended_box_vectors)
    simulation.context.setPositions(extended_positions)
    simulation.context.setVelocities(velocities)  # reuse the velocities from equilibration

    if report_production:
        production_reporter_path = "{}/production_{}.dat".format(file_path, file_name)
        simulation.reporters.append(StateDataReporter(production_reporter_path, num_steps // 50000, step=True, potentialEnergy=True, temperature=True, density=True, kineticEnergy=True, totalEnergy=True, volume=True))

    simulation.reporters.append(HDF5Reporter(path, write_out_freq))
    simulation.step(num_steps)
    logger.info("Production complete")

    return os.path.abspath(path)


In [25]:

platform_name = "CPU"  # or "CUDA", "OpenCL" depending on your hardware
file_path = "./output"
file_name = "waterbox_simulation"

if not os.path.exists(file_path):
    os.makedirs(file_path)

water_box, simulation = run_equilibration(platform_name, file_path, file_name)

In [45]:
from simtk.openmm import Vec3

def extend_box_z(positions, box_vectors, topology, factor=5):
    """ Extend the box size in the z direction by the given factor """
    new_positions = []
    new_topology = app.Topology()
    new_topology.setPeriodicBoxVectors( (
        Vec3(box_vectors[0][0], box_vectors[0][1], box_vectors[0][2]),
        Vec3(box_vectors[1][0], box_vectors[1][1], box_vectors[1][2]),
        Vec3(box_vectors[2][0], box_vectors[2][1], box_vectors[2][2]) * factor )
    )

    atom_map = {}
    for chain in topology.chains():
        new_chain = new_topology.addChain(chain.id)
        for residue in chain.residues():
            new_residue = new_topology.addResidue(residue.name, new_chain, residue.id)
            for atom in residue.atoms():
                new_atom = new_topology.addAtom(atom.name, atom.element, new_residue, atom.id)
                atom_map[atom.index] = new_atom

    for bond in topology.bonds():
        new_topology.addBond(atom_map[bond[0].index], atom_map[bond[1].index])

    for i in range(factor):
        for pos in positions:
            new_pos = pos + unit.Quantity((0, 0, i * box_vectors[2][2].value_in_unit(unit.nanometer)), unit.nanometer)
            new_positions.append(new_pos)

    new_box_vectors = (
        Vec3(box_vectors[0][0], box_vectors[0][1], box_vectors[0][2]),
        Vec3(box_vectors[1][0], box_vectors[1][1], box_vectors[1][2]),
        Vec3(box_vectors[2][0], box_vectors[2][1], box_vectors[2][2]) * factor
    )

    return new_positions, new_topology, new_box_vectors


In [37]:
def run_production(platform_name, file_path, file_name, extended_positions, velocities, extended_box_vectors, extended_topology, report_production=True, num_steps=100000, write_out_freq=1000):
    logger.info("Starting production via OpenMM")

    path = '{}/{}.h5'.format(file_path, file_name)

    platform = Platform.getPlatformByName(platform_name)

    # Load the appropriate force field for your water model
    forcefield = app.ForceField('tip3p.xml')

    system = forcefield.createSystem(extended_topology, nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer, constraints=app.AllBonds, rigidWater=False)

    thermostat = AndersenThermostat(SimulationSetup.temperature, 1 / unit.picosecond)
    system.addForce(thermostat)

    integrator = VerletIntegrator(SimulationSetup.time_step)
    simulation = Simulation(extended_topology, system, integrator, platform)
    simulation.context.setPeriodicBoxVectors(*extended_box_vectors)
    simulation.context.setPositions(extended_positions)
    simulation.context.setVelocities(velocities)  # reuse the velocities from equilibration

    if report_production:
        production_reporter_path = "{}/production_{}.dat".format(file_path, file_name)
        simulation.reporters.append(StateDataReporter(production_reporter_path, num_steps // 50000, step=True, potentialEnergy=True, temperature=True, density=True, kineticEnergy=True, totalEnergy=True, volume=True))

    simulation.reporters.append(HDF5Reporter(path, write_out_freq))
    simulation.step(num_steps)
    logger.info("Production complete")

    return os.path.abspath(path)


In [46]:
state = simulation.context.getState(getPositions=True, getVelocities=True)
positions = state.getPositions()
velocities = state.getVelocities()
box_vectors = state.getPeriodicBoxVectors()

extended_positions, extended_topology, extended_box_vectors = extend_box_z(positions, box_vectors, water_box.topology, factor=5)

In [47]:
run_production(platform_name, file_path, file_name, extended_positions, velocities, extended_box_vectors, extended_topology, report_production=True, num_steps=100000, write_out_freq=1000)

OpenMMException: Called setPositions() on a Context with the wrong number of positions